# Pytoch를 이용한 초보 code 입니다.

대회 초반 baseline 으로 올려주신 code를 
https://dacon.io/competitions/official/235626/codeshare/1538
바탕으로 시작했습니다.

In [ ]:
#Import
!pip install tensorboardX
import os
import random
import numpy as np
import pandas as pd
import cv2
from collections import defaultdict
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from datetime import datetime
from tensorboardX import SummaryWriter

Data 변형:

Test size 는 변환하면서 사용.

 초반 psuedo 만들시 train data의 30% 를 validation에 사용.
 Psuedo는 여러개 test 하였으며, 일부를 random 하게 test set으로 바꾸어서 train data에서 활용되지 않게함.
 
Psuedo 전 확인 단계에서는 train data의 30% 를 validation 로 사용.

Psuedo 이 후 확인단계에서는 train data의 30% ~ 99.9% 를 validation 로 사용.
또한 pseudo data의 100% ~ 80% 를 train data 로 사용.

최종 확인 단계에서는 psuedo의 99%를 train data로 사용.

In [ ]:
device = torch.device("cuda")
class reshape_input():
    def __init__(self,input,noise, is_pseudo=False):
        self.input = input
        self.noise = noise
        self.is_pseudo = is_pseudo
    def merge(self, list1, list2):
        merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
        return merged_list
    def  reshape_x(self):
        inputx=np.concatenate(
            [
                pd.get_dummies(self.input.letter).values.reshape(-1, 1, 26),
                (self.input[[str(j) for j in range(784)]]).values.reshape(-1, 1, 784)
            ],
            axis=2
        )

        inputy=self.input['digit'].values
        X_image=np.array(inputx[:, :, 26:], dtype="uint8")
        X_image = X_image.squeeze()

        appendimg = []
        for index in range(0, len(X_image)):
            img = X_image[index]
        # print(img.shape)
            img = np.where(img < self.noise, 0, img)
            img = img.reshape(28, 28)
            appendimg.append(img)
        appended = np.asarray(appendimg, dtype="float")
        inputxnum = appended/255.
        if self.is_pseudo:
            X_train, X_valid, xtrainl, xvalidl, y_train, y_valid = train_test_split(inputxnum, inputx, inputy,
                                                                                    test_size=0.001, random_state=42)

        else:
            X_train, X_valid, xtrainl, xvalidl, y_train, y_valid = train_test_split(inputxnum, inputx, inputy, test_size=0.99, random_state=42)

        X_train = torch.Tensor(X_train)
        xtrainl = torch.Tensor(xtrainl)
        X_valid = torch.Tensor(X_valid)
        xvalidl = torch.Tensor(xvalidl)
        y_train = torch.LongTensor(y_train)
        y_valid = torch.LongTensor(y_valid)

        return X_train, X_valid, xtrainl, xvalidl, y_train, y_valid


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
os.environ['PYTHONHASHSEED'] = str(seed_val)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


여러가지 model test: 

Model은 변환하면서 사용:

dropout 은 0.2~0.4 test 진행

conv1은 letter를 불러오는 model이니 고정.

최종 단계에서는 conv4 사용.

In [ ]:
class ConvClassifier(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, 3, padding=1), nn.ReLU(),
            nn.Conv1d(16, 64, 4, padding=1), nn.ReLU(),
            nn.Conv1d(64, 128, 5, padding=2), nn.ReLU(),
            nn.Conv1d(128, 64, 4, padding=2), nn.ReLU(),
            nn.Conv1d(64, 16, 3), nn.ReLU(),
            nn.Dropout(0.4),
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(1, 32, 3), nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3), nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 5, stride=2, padding=2), nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.4),

            nn.Conv2d(64, 64, 3), nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3), nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 5, stride=2, padding=2), nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(0.4),

            nn.Conv2d(128, 128, 4), nn.ReLU(),
            nn.BatchNorm2d(128),
        )
                                # Image를 처리할 2D Conv Block
        self.conv2 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 128, 5, padding=2), nn.ReLU(),
            nn.Conv2d(128, 256, 7, padding=3), nn.ReLU(),
            nn.Conv2d(256, 512, 9, padding=3), nn.ReLU(),
            nn.Conv2d(512, 256, 9, padding=3), nn.ReLU(),
            nn.Conv2d(256, 128, 7, padding=3), nn.ReLU(),
            nn.Conv2d(128, 64, 7, padding=3), nn.ReLU(),
            nn.Conv2d(64, 32, 5, padding=3), nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(1, 128, 5, padding=2), nn.ReLU(),  # 28
            # nn.BatchNorm2d(128),
            nn.Conv1d(128, 128, 3, padding=1), nn.ReLU(),  # 28
            # nn.BatchNorm2d(128),
            # nn.MaxPool2d(2, stride=2), #14
            nn.Conv1d(128, 256, 3, padding=1), nn.ReLU(),  # 14
            # nn.BatchNorm2d(256),
            nn.Conv1d(256, 256, 3, padding=1), nn.ReLU(),  # 14
            nn.Conv1d(256, 16, 3), nn.ReLU(),  # 26
        )
        
        self.dacon_base = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1), nn.ReLU(),  # 26
            nn.Conv2d(10, 128, 5, padding=2), nn.ReLU(),  # 26
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 5, padding=2), nn.ReLU(),  # 26
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 128, 4), nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm2d(128)

        )

        self.conv_test1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),  # 26
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 256, 5, padding=2), nn.ReLU(),  # 26
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 9, padding=3), nn.ReLU(),
            nn.Conv2d(512, 256, 9, padding=3), nn.ReLU(),
            nn.Conv2d(256, 128, 7, padding=3), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 128, 7, padding=3), nn.ReLU(),
            nn.BatchNorm2d(128)

        )

        self.out = nn.Sequential(
            nn.Linear(512, 256), nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 32), nn.ReLU(),
            nn.Linear(32, 10)
        )


    def forward(self, x1, x2, label=False):
        bsz = x1.size(0)
        x1 = self.conv1(x1)
        x2 = self.conv4(x2)
        x1 = x1.view(bsz, -1)
        x2 = x2.view(bsz, -1)
        x = torch.cat([x1, x2], dim=1)
        torchsize=x[0].shape
        out = self.out(x)

        return out

Model과 test size 외에도, 주로 많이 실험해던 변수들:

Data에서 주로 매우 높은 밝기?를 가진 pixel들이 숫자를 보여주는데 사용되었기에 낮은 밝기?의 pixel들의 noise제거 진행. 

Noise 제거는 5~15 사이로 test 진행.

In [ ]:
######### 주요 변수
batch_size = 32
noise = 10
is_pseudo_ex = True
epochs = 85
#########

In [ ]:
#log_setting
now=datetime.now()
date = now.strftime("%m%d.%Y")
time = now.strftime("%H%M")
from pathlib import Path
Path("./logs/"+date).mkdir(parents=True, exist_ok=True)
logpath="./logs/"+date
boardlogname = str(time)+".noise."+str(noise)+".pseudo."+str(is_pseudo_ex)
Path("./logsboard/"+boardlogname).mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(os.path.join('./logsboard/', boardlogname))

#call cuda
torch.cuda.is_available() #cuda
device = torch.device("cuda") #or cpu

#input 주요 input

train = pd.read_csv('../data/train.csv')
if is_pseudo_ex:
    pseudo = pd.read_csv('../data/pseudo_df.csv')
    #pseudo = pd.read_csv('./data/0903_update_20376.csv')
    pseudo.set_index('id')
test  = pd.read_csv('../data/test.csv')

FileNotFoundError: ignored

In [ ]:
train.set_index('id')
test.set_index('id')

#inputdata normalization
reshape_input_class = reshape_input(train,noise, is_pseudo=False)
X_train, X_valid, xtrainl, xvalidl, y_train, y_valid = reshape_input_class.reshape_x()
#print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)
if is_pseudo_ex:
    reshape_pseudo_class = reshape_input(pseudo, noise, is_pseudo= is_pseudo_ex)
    pX_train, pX_valid, pxtrainl, pxvalidl, py_train, py_valid = reshape_pseudo_class.reshape_x()
    print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, xvalidl.shape, xtrainl.shape)
    print(pX_train.shape, pX_valid.shape, py_train.shape, py_valid.shape, pxvalidl.shape, pxtrainl.shape)
    X_train=torch.cat([X_train, pX_train],dim=0)
    xtrainl = torch.cat([xtrainl, pxtrainl], dim=0)
    y_train = torch.cat([y_train, py_train], dim=0)
    print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, xvalidl.shape, xtrainl.shape)

#dataloader
from torch.utils.data import (
    TensorDataset, 
    DataLoader, 
    RandomSampler, 
    SequentialSampler
)

train_data = TensorDataset(
    xtrainl[:, :, :26], # Letter
    X_train.reshape(-1, 1, 28, 28), # Image (28, 28)
    y_train # Label
)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(
    train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(
    xvalidl[:, :, :26], # Letter
    X_valid.reshape(-1, 1, 28, 28), # Image (28, 28)
    y_valid # Label
)
validation_sampler = SequentialSampler(
    validation_data)
validation_dataloader = DataLoader(
    validation_data, sampler=validation_sampler, batch_size=batch_size)


model = ConvClassifier()
model.cuda()

x1 = xtrainl[:32, :, :26].cuda()
x2 = X_train[:32, :, :].reshape(-1, 1, 28, 28).cuda()
model(x1, x2).shape

# Define loss model, multi-label to cross entropy loss
ce_loss = nn.CrossEntropyLoss()

# Define optimizer and scheduler
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler
optimizer = Adam(
    model.parameters(),
    lr=1e-3, # 학습률
    eps=1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[20, 50], gamma=0.1 )


#재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
os.environ['PYTHONHASHSEED'] = str(seed_val)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device("cuda")

def flat_accuracy(probs, labels):
    pred_flat = np.argmax(probs, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

max_dict = dict()


for epoch_i in range(0, epochs):
    # 로스 초기화
    total_loss = 0
    train_accuracy = 0.0
    model.train()
    nb_train_steps = 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        x1, x2, label = batch
        logits = model(x1, x2, label)
        probs = F.softmax(logits, dim=1)
        ## forward
        loss = ce_loss(probs, label)
        # Backward 수행으로 그래디언트 계산
        loss.backward()
        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        # 그래디언트 초기화
        model.zero_grad()
        # 총 로스 계산
        total_loss += loss.item()
        # 정확도 계산
        probs = probs.detach().cpu().numpy()
        label = label.to('cpu').numpy()
        tmp_train_accuracy = flat_accuracy(probs, label)
        train_accuracy += tmp_train_accuracy
        nb_train_steps += 1
    # 평균 로스 계산
    traccu= train_accuracy/nb_train_steps
    avg_train_loss = total_loss / len(train_dataloader)
    writer.add_scalar('train/loss', avg_train_loss, epoch_i)
    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        x1, x2, label = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            logits = model(x1, x2, label)
            # label이 입력으로 들어오면 loss도 계산해서 return
            probs = F.softmax(logits, dim=1)
            # 로스 구함
            loss = ce_loss(probs, label)

            # CPU로 데이터 이동
            probs = probs.detach().cpu().numpy()
            label = label.to('cpu').numpy()

            # 출력 로짓과 라벨을 비교하여 정확도 계산
            tmp_eval_accuracy = flat_accuracy(probs, label)
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1
    lrnrate = str(optimizer.param_groups[0]['lr'])
    test_set_accuracy = eval_accuracy/nb_eval_steps
    max_dict[epoch_i]=test_set_accuracy
    s = f'\r[Epoch {epoch_i+1}/{epochs}]'
    s += " Train Acc: {0:.2f}".format(train_accuracy/nb_train_steps)
    s += f' Avg Training Loss: {avg_train_loss:.2f}'
    s += " Valid Acc: {0:.2f}".format(test_set_accuracy)
    s += " learning rate: "+lrnrate
    print(s, end='')
    scheduler.step()

    writer.add_scalar('train/score', traccu, epoch_i)
    writer.add_scalar('val/score', test_set_accuracy, epoch_i)


    torch.save(model.state_dict(), os.path.join("./modelsaves_1/epoch-{a:d}-{p:.3f}.pt".format(a=epoch_i,p=test_set_accuracy)))
#     torch.save(model, os.path.join("./modelsaves_1/epoch_model-{a:d}-{p:.3f}.pt".format(a=epoch_i,p=test_set_accuracy)))
max_val = max(max_dict.values())
def maxkey():
    max_val = max(max_dict.values())
    max_index = list(max_dict.values()).index(max_val)
    # keys = filter(lambda x: max_dict[x] == max_val, max_dict.keys())
    return max_index


print("")
print("Training complete! max accuracy: ", max_val, maxkey() )


logname=str(time)+".noise."+str(noise)+".pseudo."+str(is_pseudo_ex)+".max_accu."+str(format(max_val, '.3f')+".txt")
print(logname)
with open(os.path.join(logpath, logname), 'w') as fp:
    pass

maxepoch=str(maxkey())
max_val=float("{:.3f}".format(max_val))
maxval=str(max_val)
maxepochpt="./modelsaves_1/epoch-"+maxepoch+"-"+maxval+".pt"

print(maxepochpt)

Inference 용 data 불러오는 함수

In [ ]:
class reshape_input_inf():
    def __init__(self,input,noise):
        self.input = input
        self.noise = noise
    def merge(self, list1, list2):
        merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
        return merged_list
    def  reshape_x_inf(self):
        inputx=np.concatenate(
            [
                pd.get_dummies(self.input.letter).values.reshape(-1, 1, 26),
                (self.input[[str(j) for j in range(784)]]).values.reshape(-1, 1, 784)
            ],
            axis=2
        )

        X_image=np.array(inputx[:, :, 26:], dtype="uint8")
        X_image = X_image.squeeze()

        appendimg = []
        for index in range(0, len(X_image)):
            img = X_image[index]
        # print(img.shape)
            img = np.where(img < self.noise, 0, img)
            img = img.reshape(28, 28)
            appendimg.append(img)
        appended = np.asarray(appendimg, dtype="float")
        inputxnum = appended/255.

        X_test = torch.Tensor(inputxnum)
        X_testl = torch.Tensor(inputx)


        return X_test, X_testl


앞서 진행한 epoch 의 maximum accuracy가 저장된 model을 current model로 사용하여 inference 진행:

Psuedo를 만들때도 사용한 code로 submission시 image 정보 제거해서 제출.

In [ ]:
#same parmeter as best trained model
batch_size = 32
noise = 10
current_model = './modelsaves_1/epoch-76-0.933.pt' #예시

test  = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/submission.csv')
test.set_index('id')

from torch.utils.data import (
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler
)

modelinf = ConvClassifier()
modelinf.cuda()
modelinf.eval()
reshape_test_class = reshape_input_inf(test, noise)
X_test, X_testl = reshape_test_class.reshape_x_inf()

test_data = TensorDataset(
    X_testl[:, :, :26].cuda(),  # Letter
    X_test.reshape(-1, 1, 28, 28).cuda()
)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
torch.load(current_model)
y_pred = []
y_raw_pred = []
modelinf.load_state_dict(torch.load(current_model))

In [ ]:
#submission 혹은 pseudo data 만들때 같이 사용.
for letter, number in test_dataloader:
    with torch.no_grad():
        outputss = modelinf(letter, number)
        outputs = F.softmax(outputss, dim=1)
        y_raw_pred.append(outputs.cpu().detach().numpy())
    y_pred.append(torch.argmax(outputs, dim=1))
y_raw_pred = np.concatenate(y_raw_pred, axis=0)
pred_columns = []
for i in range(10):
   pred_columns.append("p"+str(i))

submission.digit = torch.cat(y_pred).detach().cpu().numpy()
submission2 = pd.DataFrame(data=y_raw_pred,columns=pred_columns)
submission2.insert(0,'id',value= submission.id.values)
submission2.insert(1,'digit',value= submission.digit.values)
submission2.to_csv("submission_"+str(noise)+"_"+".csv", index=False)

Baselne code에서 차이점은 noise제거 와 pseudo labelling 입니다.

Noise들은 image를 숫자로 보면 쉽게 볼 수 있었는데 제거시 도움이 많이 되었습니다.

이후 psuedo 가지고 test를 많이 했습니다. Probability를 정보를 받아와서 여러가지 cutoff로 test를 진행하기도 하였고,

import 한 psuedo data를 전부 사용하지 않고 random 하기 일부를 제거하여 test를 진행했습니다.

최종 test 중 psuedo file에서 0.1%를 제거했을때 valid accuracy가 가장 올라갔습니다.